In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import kurtosis, skew
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scikit_posthocs as sp
from scipy.stats import wilcoxon

In [3]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

filePath = os.path.join(tempTopDir, 'likertSlim_corrected.parquet')
likertSlim = pd.read_parquet(filePath)


likertSlim['visuals'] = np.where(likertSlim['visuals']=="Visuals","Charts",
                        np.where(likertSlim['visuals']=="Blur_Plus_Vis","Blur+Charts",likertSlim['visuals']))

likertSlim['ag_correctedPerc'] = likertSlim['1_2_correctedPerc']

In [4]:
# drop particpants who have not been accepted (rejected or timed out)
pID_removed1 = likertSlim[(likertSlim['status']!='APPROVED')].ParticipantPublicID.unique()
print(len(likertSlim))
# drop these given row
# indexes from dataFrame
likertSlim = likertSlim[~likertSlim['ParticipantPublicID'].isin(pID_removed1)].copy()
print(len(likertSlim))
# tmp_slimDF = slimDF[~slimDF['ParticipantPublicID'].isin(pID_removed1)].copy()
# print(tmp_slimDF.status.unique())
# print(pID_removed1)
    
# tmp_demoDF_L = demoDF_L[~demoDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_demoDF_S = demoDF_S[~demoDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_consentDF_L = consentDF_L[~consentDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_consentDF_S = consentDF_S[~consentDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_aiDF_L = aiDF_L[~aiDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_aiDF_S = aiDF_S[~aiDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_taskDF = taskDF[~taskDF['ParticipantPublicID'].isin(pID_removed1)].copy()

# print(tmp_taskDF[tmp_taskDF['ParticipantPublicID']==pID_removed1[-3]])

232
232


In [5]:
df = likertSlim.copy()

for vis in df.visuals.unique():
    print(str(vis) + " " + str(len(df[df['visuals']==vis])))

Blur+Charts 54
Charts 58
Blurred 64
Video 56


In [6]:
vis = sorted(df['visuals'].unique())
print(vis)

['Blur+Charts', 'Blurred', 'Charts', 'Video']


In [7]:
# function to make statsmodel into dataframe
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    print(dir(results))
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

In [8]:
def negative_bold(val):
    bold = 'bold' if val < 0 else ''
    return 'font-weight: %s' % bold

In [9]:
# levene:
print("Q1")
print(stats.levene(df[df['visuals']==vis[0]]['Qs1_CorrectedPerc'],\
                      df[df['visuals']==vis[1]]['Qs1_CorrectedPerc'],\
                         df[df['visuals']==vis[2]]['Qs1_CorrectedPerc'],\
                            df[df['visuals']==vis[3]]['Qs1_CorrectedPerc']))

# shapiro:
print(stats.shapiro(df['Qs1_CorrectedPerc']))

# levene:
print("Q2")
print(stats.levene(df[df['visuals']==vis[0]]['Qs2_CorrectedPerc'],\
                      df[df['visuals']==vis[1]]['Qs2_CorrectedPerc'],\
                         df[df['visuals']==vis[2]]['Qs2_CorrectedPerc'],\
                            df[df['visuals']==vis[3]]['Qs2_CorrectedPerc']))

# shapiro:
print(stats.shapiro(df['Qs2_CorrectedPerc']))

# levene:
print("Q1&2")
print(stats.levene(df[df['visuals']==vis[0]]['ag_correctedPerc'],\
                      df[df['visuals']==vis[1]]['ag_correctedPerc'],\
                         df[df['visuals']==vis[2]]['ag_correctedPerc'],\
                            df[df['visuals']==vis[3]]['ag_correctedPerc']))

# shapiro:
print(stats.shapiro(df['ag_correctedPerc']))

# levene:
print("1_2 no G")
print(stats.levene(df[df['visuals']==vis[0]]['1_2_WO_Ghosts_correctedPerc'],\
                      df[df['visuals']==vis[1]]['1_2_WO_Ghosts_correctedPerc'],\
                         df[df['visuals']==vis[2]]['1_2_WO_Ghosts_correctedPerc'],\
                            df[df['visuals']==vis[3]]['1_2_WO_Ghosts_correctedPerc']))

# shapiro:
print(stats.shapiro(df['1_2_WO_Ghosts_correctedPerc']))

# levene:
print("Comp")
print(stats.levene(df[df['visuals']==vis[0]]['comp_correctedPerc'],\
                      df[df['visuals']==vis[1]]['comp_correctedPerc'],\
                         df[df['visuals']==vis[2]]['comp_correctedPerc'],\
                            df[df['visuals']==vis[3]]['comp_correctedPerc']))

# shapiro:
print(stats.shapiro(df['comp_correctedPerc']))

# levene:
print("Tot no G")
print(stats.levene(df[df['visuals']==vis[0]]['tot_WO_Ghosts_correctedPerc'],\
                      df[df['visuals']==vis[1]]['tot_WO_Ghosts_correctedPerc'],\
                         df[df['visuals']==vis[2]]['tot_WO_Ghosts_correctedPerc'],\
                            df[df['visuals']==vis[3]]['tot_WO_Ghosts_correctedPerc']))

# shapiro:
print(stats.shapiro(df['tot_WO_Ghosts_correctedPerc']))

Q1
LeveneResult(statistic=3.2435817915280842, pvalue=0.022809424342174395)
ShapiroResult(statistic=0.9279556274414062, pvalue=3.2131235450094664e-09)
Q2
LeveneResult(statistic=5.724973157833499, pvalue=0.0008561317090585818)
ShapiroResult(statistic=0.9419251680374146, pvalue=5.61457156322831e-08)
Q1&2
LeveneResult(statistic=3.121314193296522, pvalue=0.02678579960714649)
ShapiroResult(statistic=0.9522904753684998, pvalue=6.176475721986208e-07)
1_2 no G
LeveneResult(statistic=4.173247372429688, pvalue=0.006682882184632748)
ShapiroResult(statistic=0.950961172580719, pvalue=4.4710751012644323e-07)
Comp
LeveneResult(statistic=0.8319227182650887, pvalue=0.4775297119747508)
ShapiroResult(statistic=0.7859591245651245, pvalue=4.064780419336604e-17)
Tot no G
LeveneResult(statistic=1.8012506803322499, pvalue=0.14773955553849202)
ShapiroResult(statistic=0.9773411154747009, pvalue=0.0008922450942918658)


In [10]:
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
# Part Two
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################

def fancyWilcoxon(a,b,df):
    data1 = df[a]
    data2 = df[b]
    print("\n" + str(a) + ' ' + str(stats.tmean(df[a])) + ' ' + str(stats.tstd(df[a])))
    print(str(b) + ' ' + str(stats.tmean(df[b])) + ' ' + str(stats.tstd(df[b])))
    # Calculate the Wilcoxon signed-rank test.
    return(wilcoxon(data1, data2))

In [11]:
# Part Two:

df = likertSlim.copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot2Perc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
a = 'Qs2_WO_Ghosts_CorrectedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

a = 'Qs2_CorrectedPerc'
b = 'Qs1_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

# Part Two:

#(Wilcoxon) rank sum test: Unpaired Samples, extended by Mann and Whitney
#(Wilcoxon) signed rank test: Paired Samples, corresponds to 1-sample t-test on paired differences




Not Corrected

tot2Perc 28.125 18.45456278121721
tot1Perc 27.747844827586206 18.386424377668725
WilcoxonResult(statistic=7037.0, pvalue=0.92133477789629)

Corrected

Qs2_WO_Ghosts_CorrectedPerc 25.45925697865353 19.964350734729734
Qs1_WO_Ghosts_CorrectedPerc 25.338669950738915 19.60697955628808
WilcoxonResult(statistic=10137.0, pvalue=0.9869500556724824)

Qs2_CorrectedPerc 23.792718596059117 17.536737754032146
Qs1_CorrectedPerc 23.486761083743847 17.946894290361623
WilcoxonResult(statistic=10795.0, pvalue=0.7486110793055875)


In [12]:
# Part Two:

df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("ALL")
print("\nNot Corrected")

# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

# Note on ranksums():
#This test should be used to compare two samples from continuous distributions. It
# does not handle ties between measurements in x and y. For tie-handling and an 
# optional continuity correction see scipy.stats.mannwhitneyu.
# print(stats.mannwhitneyu(data1, data2))

# df = likertSlim.copy()
df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nEASY")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

# Part Two:

# df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nDIFFICULT")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


ALL

Not Corrected

compPerc 64.22413793103448 25.5299115653581
tot1Perc 27.747844827586206 18.386424377668725
WilcoxonResult(statistic=762.5, pvalue=8.128151081279301e-32)

Corrected

comp_correctedPerc 37.93103448275862 36.99812189873208
Qs1_CorrectedPerc 23.486761083743847 17.946894290361623
WilcoxonResult(statistic=7885.5, pvalue=1.8205231559593974e-06)

Corrected

comp_correctedPerc 37.93103448275862 36.99812189873208
Qs1_WO_Ghosts_CorrectedPerc 25.338669950738915 19.60697955628808
WilcoxonResult(statistic=8355.0, pvalue=3.9409510680132806e-05)

EASY

Not Corrected

compPerc 70.45454545454545 23.748930005307553
tot1Perc 26.136363636363637 18.43547003639684
WilcoxonResult(statistic=68.5, pvalue=1.639391480090377e-17)

Corrected

comp_correctedPerc 46.36363636363637 36.28954085747533
Qs1_CorrectedPerc 22.047889610389618 17.608688002242793
WilcoxonResult(statistic=1129.5, pvalue=7.524159780632649e-08)

Corrected

comp_correctedPerc 46.36363636363637 36.28954085747533
Qs1_WO_Ghosts_Co

In [13]:
# Part Two:

df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nALL")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

a = 'comp_correctedPerc'
b = 'Qs2_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


# df = likertSlim.copy()
df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nEASY")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

a = 'comp_correctedPerc'
b = 'Qs2_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

    
# df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nDIFFICULT")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

a = 'comp_correctedPerc'
b = 'Qs2_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))



ALL

Not Corrected

compPerc 64.22413793103448 25.5299115653581
tot2Perc 28.125 18.45456278121721
WilcoxonResult(statistic=931.0, pvalue=1.4970100086293387e-32)

Corrected

comp_correctedPerc 37.93103448275862 36.99812189873208
Qs2_CorrectedPerc 23.792718596059117 17.536737754032146
WilcoxonResult(statistic=7599.0, pvalue=2.1739613846391795e-06)

EASY

Not Corrected

compPerc 70.45454545454545 23.748930005307553
tot2Perc 26.818181818181817 17.41924055801558
WilcoxonResult(statistic=66.0, pvalue=2.97995579796143e-18)

Corrected

comp_correctedPerc 46.36363636363637 36.28954085747533
Qs2_CorrectedPerc 22.531655844155846 16.32590141410491
WilcoxonResult(statistic=1045.5, pvalue=2.8014077355346938e-08)

DIFFICULT

Not Corrected

compPerc 58.60655737704918 25.867289205873828
tot2Perc 29.30327868852459 19.336254367457343
WilcoxonResult(statistic=480.0, pvalue=1.8339164257413177e-15)

Corrected

comp_correctedPerc 30.327868852459016 36.11127033223079
Qs2_CorrectedPerc 24.929742388758783 18.5

In [14]:
# Parts One and Two:

df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nALL")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

    
# df = likertSlim.copy()
df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nEASY")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

    
# df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("\nDIFFICULT")
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


ALL

Not Corrected

compPerc 64.22413793103448 25.5299115653581
agTotPerc 27.936422413793103 15.21951565728815
WilcoxonResult(statistic=802.5, pvalue=3.3306648588766596e-34)

Corrected

comp_correctedPerc 37.93103448275862 36.99812189873208
1_2_correctedPerc 23.63973983990148 14.57156298442052
WilcoxonResult(statistic=8351.0, pvalue=1.5942166583344882e-06)

EASY

Not Corrected

compPerc 70.45454545454545 23.748930005307553
agTotPerc 26.477272727272727 14.215112833203868
WilcoxonResult(statistic=64.5, pvalue=1.5461738979264189e-18)

Corrected

comp_correctedPerc 46.36363636363637 36.28954085747533
1_2_correctedPerc 22.28977272727272 13.25234468438674
WilcoxonResult(statistic=1163.0, pvalue=2.9184691570567482e-08)

DIFFICULT

Not Corrected

compPerc 58.60655737704918 25.867289205873828
agTotPerc 29.25204918032787 16.01560112007135
WilcoxonResult(statistic=414.0, pvalue=8.385867108706913e-17)

Corrected

comp_correctedPerc 30.327868852459016 36.11127033223079
1_2_correctedPerc 24.8569233

In [15]:

print(df['Qs1_CorrectedPerc'].mean())
print(df['Qs1_CorrectedPerc'].std())
print()
print(df['Qs2_CorrectedPerc'].mean())
print(df['Qs2_CorrectedPerc'].std())
print()
print(df['1_2_correctedPerc'].mean())
print(df['1_2_correctedPerc'].std())
print()
print(df['comp_correctedPerc'].mean())
print(df['comp_correctedPerc'].std())
print()

24.78410421545668
18.2210407185097

24.929742388758783
18.55438959543301

24.85692330210773
15.62050865045431

30.327868852459016
36.11127033223079



In [16]:
# Part Two (charts):

df = likertSlim[likertSlim['visuals']=="Charts"]

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 40.30172413793103 19.596128255072014
tot2Perc 41.5948275862069 16.784876467818503
WilcoxonResult(statistic=412.5, pvalue=0.6192802178301384)

Corrected

Qs1_CorrectedPerc 35.250923645320206 19.599736625242876
Qs2_CorrectedPerc 36.60406403940887 16.490192791783375
WilcoxonResult(statistic=713.0, pvalue=0.7994744465371784)

Not Corrected

compPerc 68.96551724137932 24.472659658900977
tot1Perc 40.30172413793103 19.596128255072014
WilcoxonResult(statistic=46.5, pvalue=2.2481742989336563e-08)

Corrected

Qs1_CorrectedPerc 35.250923645320206 19.599736625242876
comp_correctedPerc 44.827586206896555 35.896604897875676
WilcoxonResult(statistic=599.0, pvalue=0.10449204664728348)

WO_Ghosts

Corrected

comp_correctedPerc 44.827586206896555 35.896604897875676
Qs1_WO_Ghosts_CorrectedPerc 31.59893267651888 20.933576851755006
WilcoxonResult(statistic=558.0, pvalue=0.05016102384036066)

Not Corrected

compPerc 68.96551724137932 24.472659658900977
tot2Perc 41.5948275862069 16.7

In [17]:
# Part Two (charts):

df = likertSlim[likertSlim['visuals']=="Charts"]

df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 37.96296296296296 20.650910729556553
tot2Perc 35.648148148148145 16.519111783962664
WilcoxonResult(statistic=82.5, pvalue=0.6104284512295916)

Corrected

Qs1_CorrectedPerc 32.589285714285715 19.904907865721388
Qs2_CorrectedPerc 30.625000000000007 15.961486743594222
WilcoxonResult(statistic=138.0, pvalue=0.5097549306564958)

Not Corrected

compPerc 64.81481481481481 24.277303482624767
tot1Perc 37.96296296296296 20.650910729556553
WilcoxonResult(statistic=9.0, pvalue=0.0001934734598457642)

Corrected

Qs1_CorrectedPerc 32.589285714285715 19.904907865721388
comp_correctedPerc 37.03703703703704 32.79479494521311
WilcoxonResult(statistic=156.0, pvalue=0.6203034190834849)

WO_Ghosts

Corrected

comp_correctedPerc 37.03703703703704 32.79479494521311
Qs1_WO_Ghosts_CorrectedPerc 29.872134038800706 21.85737842546266
WilcoxonResult(statistic=146.0, pvalue=0.45336618657282424)

Not Corrected

compPerc 64.81481481481481 24.277303482624767
tot2Perc 35.648148148148145 16.5191

In [18]:
# Part Two (charts):

df = likertSlim[likertSlim['visuals']=="Charts"]

# df = df[df['difficulty']=="Easy"].copy()
df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 42.33870967741935 18.72982785860306
tot2Perc 46.774193548387096 15.46762140175266
WilcoxonResult(statistic=96.5, pvalue=0.19595204475736638)

Corrected

Qs1_CorrectedPerc 37.569124423963146 19.354000437876113
Qs2_CorrectedPerc 41.81163594470048 15.351331153025415
WilcoxonResult(statistic=174.5, pvalue=0.3523914224276794)

Not Corrected

compPerc 72.58064516129032 24.456456807352378
tot1Perc 42.33870967741935 18.72982785860306
WilcoxonResult(statistic=15.5, pvalue=2.698628881935496e-05)

Corrected

Qs1_CorrectedPerc 37.569124423963146 19.354000437876113
comp_correctedPerc 51.61290322580645 37.60290540136035
WilcoxonResult(statistic=147.0, pvalue=0.0785676254945455)

WO_Ghosts

Corrected

comp_correctedPerc 51.61290322580645 37.60290540136035
Qs1_WO_Ghosts_CorrectedPerc 33.102918586789556 20.33571078913146
WilcoxonResult(statistic=141.0, pvalue=0.05971400135462979)

Not Corrected

compPerc 72.58064516129032 24.456456807352378
tot2Perc 46.774193548387096 15.467621

In [19]:
# Part Two (blur + charts):

df = likertSlim[likertSlim['visuals']=="Blur+Charts"]

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 24.305555555555557 17.580117593212936
tot2Perc 30.09259259259259 19.80424818017162
WilcoxonResult(statistic=297.0, pvalue=0.07848710304533774)

Corrected

Qs1_CorrectedPerc 20.120701058201057 17.412793239341244
Qs2_CorrectedPerc 26.03174603174603 18.875675170530208
WilcoxonResult(statistic=425.5, pvalue=0.0627935535398971)

Not Corrected

compPerc 66.20370370370371 24.358470402785606
tot1Perc 24.305555555555557 17.580117593212936
WilcoxonResult(statistic=30.0, pvalue=1.7140536442286149e-09)

Corrected

Qs1_CorrectedPerc 20.120701058201057 17.412793239341244
comp_correctedPerc 39.81481481481482 36.83162816091528
WilcoxonResult(statistic=312.0, pvalue=0.00166952408897916)

WO_Ghosts

Corrected

comp_correctedPerc 39.81481481481482 36.83162816091528
Qs1_WO_Ghosts_CorrectedPerc 25.71649029982363 22.37101549040098
WilcoxonResult(statistic=432.0, pvalue=0.030272351201763095)

Not Corrected

compPerc 66.20370370370371 24.358470402785606
tot2Perc 30.09259259259259 19.8

In [20]:
# Part Two (blur + charts):

df = likertSlim[likertSlim['visuals']=="Blur+Charts"]

df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 23.557692307692307 19.146449761288345
tot2Perc 29.807692307692307 19.389727653103808
WilcoxonResult(statistic=64.0, pvalue=0.20091031610912535)

Corrected

Qs1_CorrectedPerc 19.907280219780223 18.96484867285944
Qs2_CorrectedPerc 25.467032967032967 18.471491197211602
WilcoxonResult(statistic=104.0, pvalue=0.3009457071428284)

Not Corrected

compPerc 75.0 18.708286933869708
tot1Perc 23.557692307692307 19.146449761288345
WilcoxonResult(statistic=0.0, pvalue=7.401444819813692e-06)

Corrected

Qs1_CorrectedPerc 19.907280219780223 18.96484867285944
comp_correctedPerc 50.0 37.416573867739416
WilcoxonResult(statistic=44.0, pvalue=0.001418183123896774)

WO_Ghosts

Corrected

comp_correctedPerc 50.0 37.416573867739416
Qs1_WO_Ghosts_CorrectedPerc 24.748168498168504 24.05891594442115
WilcoxonResult(statistic=76.0, pvalue=0.019756995940741637)

Not Corrected

compPerc 75.0 18.708286933869708
tot2Perc 29.807692307692307 19.389727653103808
WilcoxonResult(statistic=0.0, pvalue

In [21]:
# Part Two (blur + charts):

df = likertSlim[likertSlim['visuals']=="Blur+Charts"]

# df = df[df['difficulty']=="Easy"].copy()
df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 25.0 16.315750172876015
tot2Perc 30.357142857142858 20.53355751269088
WilcoxonResult(statistic=91.5, pvalue=0.25026503771344977)

Corrected

Qs1_CorrectedPerc 20.318877551020403 16.18866025813936
Qs2_CorrectedPerc 26.55612244897959 19.567110274022543
WilcoxonResult(statistic=120.0, pvalue=0.15856069707732467)

Not Corrected

compPerc 58.035714285714285 26.399329597067478
tot1Perc 25.0 16.315750172876015
WilcoxonResult(statistic=18.0, pvalue=5.864350735287437e-05)

Corrected

Qs1_CorrectedPerc 20.318877551020403 16.18866025813936
comp_correctedPerc 30.357142857142858 34.26283407632531
WilcoxonResult(statistic=126.0, pvalue=0.32561124416545284)

WO_Ghosts

Corrected

comp_correctedPerc 30.357142857142858 34.26283407632531
Qs1_WO_Ghosts_CorrectedPerc 26.615646258503407 21.087736905183
WilcoxonResult(statistic=169.0, pvalue=0.8687707243203414)

Not Corrected

compPerc 58.035714285714285 26.399329597067478
tot2Perc 30.357142857142858 20.53355751269088
WilcoxonResult

In [22]:
# Part Two (video):

df = likertSlim[likertSlim['visuals']=="Video"]

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 26.785714285714285 15.342412143780077
tot2Perc 22.321428571428573 15.39522611113546
WilcoxonResult(statistic=317.0, pvalue=0.05203305116722504)

Corrected

Qs1_CorrectedPerc 22.46811224489796 14.847920977527123
Qs2_CorrectedPerc 17.96875 13.930613507480754
WilcoxonResult(statistic=441.0, pvalue=0.02388147968910955)

Not Corrected

compPerc 63.839285714285715 26.50885051910316
tot1Perc 26.785714285714285 15.342412143780077
WilcoxonResult(statistic=50.0, pvalue=7.949596279965549e-09)

Corrected

Qs1_CorrectedPerc 22.46811224489796 14.847920977527123
comp_correctedPerc 37.5 39.60027548113546
WilcoxonResult(statistic=501.5, pvalue=0.015551699236784528)

WO_Ghosts

Corrected

comp_correctedPerc 37.5 39.60027548113546
Qs1_WO_Ghosts_CorrectedPerc 25.55272108843537 15.560400740588394
WilcoxonResult(statistic=542.0, pvalue=0.055924150154050445)

Not Corrected

compPerc 63.839285714285715 26.50885051910316
tot2Perc 22.321428571428573 15.39522611113546
WilcoxonResult(stat

In [23]:
# Part Two (video):

df = likertSlim[likertSlim['visuals']=="Video"]

df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 25.862068965517242 13.746360915662201
tot2Perc 21.982758620689655 15.901191980601357
WilcoxonResult(statistic=122.0, pvalue=0.26383434042814446)

Corrected

Qs1_CorrectedPerc 21.619458128078815 13.392128469504076
Qs2_CorrectedPerc 18.251231527093598 14.39630577499226
WilcoxonResult(statistic=161.0, pvalue=0.33871370518195)

Not Corrected

compPerc 74.13793103448276 20.575165574635
tot1Perc 25.862068965517242 13.746360915662201
WilcoxonResult(statistic=0.0, pvalue=3.288275995093385e-06)

Corrected

Qs1_CorrectedPerc 21.619458128078815 13.392128469504076
comp_correctedPerc 50.0 37.79644730092272
WilcoxonResult(statistic=65.0, pvalue=0.0009700189416656473)

WO_Ghosts

Corrected

comp_correctedPerc 50.0 37.79644730092272
Qs1_WO_Ghosts_CorrectedPerc 24.91789819376026 14.45278942676475
WilcoxonResult(statistic=67.5, pvalue=0.002012158060542192)

Not Corrected

compPerc 74.13793103448276 20.575165574635
tot2Perc 21.982758620689655 15.901191980601357
WilcoxonResult(sta

In [24]:
# Part Two (video):

df = likertSlim[likertSlim['visuals']=="Video"]

# df = df[df['difficulty']=="Easy"].copy()
df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 27.77777777777778 17.101694323018076
tot2Perc 22.685185185185187 15.126484763835295
WilcoxonResult(statistic=46.0, pvalue=0.07405850623426727)

Corrected

Qs1_CorrectedPerc 23.37962962962963 16.478712595758015
Qs2_CorrectedPerc 17.665343915343918 13.679727658030886
WilcoxonResult(statistic=61.5, pvalue=0.011410723281184167)

Not Corrected

compPerc 52.77777777777778 28.02242691589025
tot1Perc 27.77777777777778 17.101694323018076
WilcoxonResult(statistic=34.0, pvalue=0.0014873184403540174)

Corrected

Qs1_CorrectedPerc 23.37962962962963 16.478712595758015
comp_correctedPerc 24.074074074074073 37.64809314399493
WilcoxonResult(statistic=179.0, pvalue=0.8100557876741423)

WO_Ghosts

Corrected

comp_correctedPerc 24.074074074074073 37.64809314399493
Qs1_WO_Ghosts_CorrectedPerc 26.234567901234573 16.9205914429324
WilcoxonResult(statistic=154.5, pvalue=0.40658910769530077)

Not Corrected

compPerc 52.77777777777778 28.02242691589025
tot2Perc 22.685185185185187 15.1264

In [25]:
# Part Two (blurred):

df = likertSlim[likertSlim['visuals']=="Blurred"]

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 20.1171875 14.539421558847279
tot2Perc 19.3359375 13.350005852058642
WilcoxonResult(statistic=417.0, pvalue=0.6596487862831149)

Corrected

Qs1_CorrectedPerc 16.556919642857146 14.108944557225726
Qs2_CorrectedPerc 15.389229910714286 12.45634756369402
WilcoxonResult(statistic=686.5, pvalue=0.48401627559827787)

Not Corrected

compPerc 58.59375 26.056543369774747
tot1Perc 20.1171875 14.539421558847279
WilcoxonResult(statistic=65.5, pvalue=5.469069106112869e-10)

Corrected

Qs1_CorrectedPerc 16.556919642857146 14.108944557225726
comp_correctedPerc 30.46875 35.205955822798295
WilcoxonResult(statistic=592.0, pvalue=0.011066680484432654)

WO_Ghosts

Corrected

comp_correctedPerc 30.46875 35.205955822798295
Qs1_WO_Ghosts_CorrectedPerc 19.159226190476193 17.483452964961337
WilcoxonResult(statistic=606.0, pvalue=0.035031374372517306)

Not Corrected

compPerc 58.59375 26.056543369774747
tot2Perc 19.3359375 13.350005852058642
WilcoxonResult(statistic=35.0, pvalue=5.145200

In [26]:
# Part Two (blurred):

df = likertSlim[likertSlim['visuals']=="Blurred"]

df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 17.410714285714285 14.16637488028173
tot2Perc 20.535714285714285 14.115219374470168
WilcoxonResult(statistic=97.0, pvalue=0.5101414606264869)

Corrected

Qs1_CorrectedPerc 14.314413265306124 13.169994358402805
Qs2_CorrectedPerc 16.434948979591837 12.95382539496097
WilcoxonResult(statistic=128.0, pvalue=0.7608261987342645)

Not Corrected

compPerc 67.85714285714286 29.546842014263945
tot1Perc 17.410714285714285 14.16637488028173
WilcoxonResult(statistic=7.5, pvalue=1.8689807608234472e-05)

Corrected

Qs1_CorrectedPerc 14.314413265306124 13.169994358402805
comp_correctedPerc 48.214285714285715 37.22340675679768
WilcoxonResult(statistic=35.5, pvalue=0.00037326288484682)

WO_Ghosts

Corrected

comp_correctedPerc 48.214285714285715 37.22340675679768
Qs1_WO_Ghosts_CorrectedPerc 15.752551020408164 14.916168333033285
WilcoxonResult(statistic=30.0, pvalue=0.00035762302970086514)

Not Corrected

compPerc 67.85714285714286 29.546842014263945
tot2Perc 20.535714285714285 14

In [27]:
# Part Two (blurred):

df = likertSlim[likertSlim['visuals']=="Blurred"]

# df = df[df['difficulty']=="Easy"].copy()
df = df[df['difficulty']=="Difficult"].copy()

# df = df[df['difficulty']=="Easy"].copy()
# df = df[df['difficulty']=="Difficult"].copy()
print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'tot1Perc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'Qs2_CorrectedPerc'
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'compPerc'
b = 'tot1Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs1_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs1_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on tot2Perc v compPerc
a = 'compPerc'
b = 'tot2Perc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on tot1Perc v tot2Perc
a = 'Qs2_CorrectedPerc'
b = 'comp_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = 'Qs2_WO_Ghosts_CorrectedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))

print("\nNot Corrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'compPerc'
b = 'agTotPerc'
print(fancyWilcoxon(a,b,df))

print("\nCorrected")
# Wilcoxon Signed-Rank Test on agTotPerc v compPerc
a = 'comp_correctedPerc'
b = '1_2_correctedPerc'
print(fancyWilcoxon(a,b,df))

print("\nWO_Ghosts")
# Wilcoxon Signed-Rank Test on tot1Perc v compPerc
a = 'comp_correctedPerc'
b = '1_2_WO_Ghosts_correctedPerc'
print("\nCorrected")
print(fancyWilcoxon(a,b,df))


Not Corrected

tot1Perc 22.22222222222222 14.672347384715842
tot2Perc 18.40277777777778 12.847356284156811
WilcoxonResult(statistic=79.0, pvalue=0.19695752376815345)

Corrected

Qs1_CorrectedPerc 18.30109126984127 14.743022978506973
Qs2_CorrectedPerc 14.575892857142854 12.17731034750953
WilcoxonResult(statistic=206.0, pvalue=0.2779054615172245)

Not Corrected

compPerc 51.388888888888886 20.653989967735086
tot1Perc 22.22222222222222 14.672347384715842
WilcoxonResult(statistic=29.0, pvalue=6.264629924203411e-06)

Corrected

Qs1_CorrectedPerc 18.30109126984127 14.743022978506973
comp_correctedPerc 16.666666666666668 26.726124191242434
WilcoxonResult(statistic=299.0, pvalue=0.7930395905100341)

WO_Ghosts

Corrected

comp_correctedPerc 16.666666666666668 26.726124191242434
Qs1_WO_Ghosts_CorrectedPerc 21.808862433862437 19.028026050650336
WilcoxonResult(statistic=242.0, pvalue=0.3417216104644608)

Not Corrected

compPerc 51.388888888888886 20.653989967735086
tot2Perc 18.40277777777778 12.8

In [28]:
# The Context of Types Of Questions:

df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

vis = sorted(df.visuals.unique())

# Kruskal-Wallis on ALL compPerc across visuals
print("ALL 1 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['Qs1_CorrectedPerc'],
               df[df['visuals']==vis[1]]['Qs1_CorrectedPerc'],
               df[df['visuals']==vis[2]]['Qs1_CorrectedPerc'],
               df[df['visuals']==vis[3]]['Qs1_CorrectedPerc'])))

print("ALL 2 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['Qs2_CorrectedPerc'],
               df[df['visuals']==vis[1]]['Qs2_CorrectedPerc'],
               df[df['visuals']==vis[2]]['Qs2_CorrectedPerc'],
               df[df['visuals']==vis[3]]['Qs2_CorrectedPerc'])))

print("ALL 1_2 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['1_2_correctedPerc'],
               df[df['visuals']==vis[1]]['1_2_correctedPerc'],
               df[df['visuals']==vis[2]]['1_2_correctedPerc'],
               df[df['visuals']==vis[3]]['1_2_correctedPerc'])))

print("ALL compPerc across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['comp_correctedPerc'],
               df[df['visuals']==vis[1]]['comp_correctedPerc'],
               df[df['visuals']==vis[2]]['comp_correctedPerc'],
               df[df['visuals']==vis[3]]['comp_correctedPerc'])))

print("ALL Perc across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['tot_correctedPerc'],
               df[df['visuals']==vis[1]]['tot_correctedPerc'],
               df[df['visuals']==vis[2]]['tot_correctedPerc'],
               df[df['visuals']==vis[3]]['tot_correctedPerc'])))

df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# Kruskal-Wallis on Easy compPerc across visuals
print("\nEASY compPerc across visual")
# print("And kw: " + \
# str(stats.kruskal(df[df['visuals']==vis[0]]['compPerc'],
#                df[df['visuals']==vis[1]]['compPerc'],
#                df[df['visuals']==vis[2]]['compPerc'],
#                df[df['visuals']==vis[3]]['compPerc'])))

print("ALL compPerc across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['comp_correctedPerc'],
               df[df['visuals']==vis[1]]['comp_correctedPerc'],
               df[df['visuals']==vis[2]]['comp_correctedPerc'],
               df[df['visuals']==vis[3]]['comp_correctedPerc'])))

df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()
# Kruskal-Wallis on Difficult compPerc across visuals
print("\nDIFFICULT compPerc across visual")
# print("And kw: " + \
# str(stats.kruskal(df[df['visuals']==vis[0]]['compPerc'],
#                df[df['visuals']==vis[1]]['compPerc'],
#                df[df['visuals']==vis[2]]['compPerc'],
#                df[df['visuals']==vis[3]]['compPerc'])))

print("ALL compPerc across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['comp_correctedPerc'],
               df[df['visuals']==vis[1]]['comp_correctedPerc'],
               df[df['visuals']==vis[2]]['comp_correctedPerc'],
               df[df['visuals']==vis[3]]['comp_correctedPerc'])))

print("ALL allPerc across visual WO Ghosts")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['tot_WO_Ghosts_correctedPerc'],
               df[df['visuals']==vis[1]]['tot_WO_Ghosts_correctedPerc'],
               df[df['visuals']==vis[2]]['tot_WO_Ghosts_correctedPerc'],
               df[df['visuals']==vis[3]]['tot_WO_Ghosts_correctedPerc'])))
print(sp.posthoc_mannwhitney(df, \
                 val_col='tot_WO_Ghosts_correctedPerc', group_col='visuals',p_adjust='holm'))

ALL 1 across visual
And kw: KruskalResult(statistic=32.62803322814898, pvalue=3.858286595862683e-07)
ALL 2 across visual
And kw: KruskalResult(statistic=49.11672081478054, pvalue=1.2319156340994957e-10)
ALL 1_2 across visual
And kw: KruskalResult(statistic=57.018373018566834, pvalue=2.5467767646976326e-12)
ALL compPerc across visual
And kw: KruskalResult(statistic=5.154321204868742, pvalue=0.16083953484349553)
ALL Perc across visual
And kw: KruskalResult(statistic=52.777066540925404, pvalue=2.045385095366996e-11)

EASY compPerc across visual
ALL compPerc across visual
And kw: KruskalResult(statistic=2.3324238778567343, pvalue=0.50633782649983)

DIFFICULT compPerc across visual
ALL compPerc across visual
And kw: KruskalResult(statistic=16.79383878079045, pvalue=0.0007791977008972442)
ALL allPerc across visual WO Ghosts
And kw: KruskalResult(statistic=29.600459848900087, pvalue=1.6746451751155045e-06)
             Blur+Charts   Blurred    Charts     Video
Blur+Charts     1.000000  0.0181

In [29]:
# Kruskal-Wallis on compPerc  by Difficulty across visuals
print("\ncompPerc across visual")
vis = likertSlim.visuals.unique()
df1 = likertSlim[likertSlim['difficulty']=="Easy"].copy()
df2 = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

print("And kw: " + \
str(stats.kruskal(df1[df1['visuals']==vis[0]]['compPerc'],
               df1[df1['visuals']==vis[1]]['compPerc'],
               df1[df1['visuals']==vis[2]]['compPerc'],
               df1[df1['visuals']==vis[3]]['compPerc'],
                 df2[df2['visuals']==vis[0]]['compPerc'],
               df2[df2['visuals']==vis[1]]['compPerc'],
               df2[df2['visuals']==vis[2]]['compPerc'],
               df2[df2['visuals']==vis[3]]['compPerc'])))

print(sp.posthoc_mannwhitney(df, \
                 val_col='compPerc', group_col='visuals',p_adjust='holm'))



print("And kw: " + \
str(stats.kruskal(df1[df1['visuals']==vis[0]]['comp_correctedPerc'],
               df1[df1['visuals']==vis[1]]['comp_correctedPerc'],
               df1[df1['visuals']==vis[2]]['comp_correctedPerc'],
               df1[df1['visuals']==vis[3]]['comp_correctedPerc'],
                 df2[df2['visuals']==vis[0]]['comp_correctedPerc'],
               df2[df2['visuals']==vis[1]]['comp_correctedPerc'],
               df2[df2['visuals']==vis[2]]['comp_correctedPerc'],
               df2[df2['visuals']==vis[3]]['comp_correctedPerc'])))

print(sp.posthoc_mannwhitney(df, \
                 val_col='comp_correctedPerc', group_col='visuals',p_adjust='holm'))


compPerc across visual
And kw: KruskalResult(statistic=31.227176597204625, pvalue=5.64526570557356e-05)
             Blur+Charts   Blurred    Charts     Video
Blur+Charts     1.000000  0.690611  0.124712  0.825072
Blurred         0.690611  1.000000  0.002269  0.965115
Charts          0.124712  0.002269  1.000000  0.035295
Video           0.825072  0.965115  0.035295  1.000000
And kw: KruskalResult(statistic=30.18005004520431, pvalue=8.800303203352669e-05)
             Blur+Charts   Blurred    Charts     Video
Blur+Charts     1.000000  0.275822  0.119403  0.718280
Blurred         0.275822  1.000000  0.000639  0.718280
Charts          0.119403  0.000639  1.000000  0.028243
Video           0.718280  0.718280  0.028243  1.000000


In [30]:
def fancyMWU(a,b,c,d):
    print("\n")
    print(str(c) + " " + str(stats.tmean(a)) + " " + str(stats.tstd(a)))
    print(str(d) + " " + str(stats.tmean(b)) + " " + str(stats.tstd(b)))
    return(stats.mannwhitneyu(a, b))

In [31]:
# Part Two:

df1 = likertSlim[likertSlim['difficulty']=="Difficult"].copy()
df2 = likertSlim[likertSlim['difficulty']=="Easy"].copy()

# No reason to pair-wise test if no sig found
# Mann Whitney U pair-wise on Difficult compPerc across visuals
# print("\nDIFFICULT")
# Mann-Whitney
print(sp.posthoc_mannwhitney(df, \
                 val_col='comp_correctedPerc', group_col='visuals',p_adjust='holm'))

# a = df1[df1['visuals']==vis[0]]['compPerc']
# b = df2[df2['visuals']==vis[1]]['compPerc']
# c = vis[0]
# d = vis[1]
# print(fancyMWU(a,b,c,d))

# a = df1[df1['visuals']==vis[0]]['compPerc']
# b = df2[df2['visuals']==vis[2]]['compPerc']
# c = vis[0]
# d = vis[2]
# print(fancyMWU(a,b,c,d))

# a = df1[df1['visuals']==vis[0]]['compPerc']
# b = df2[df2['visuals']==vis[3]]['compPerc']
# c = vis[0]
# d = vis[3]
# print(fancyMWU(a,b,c,d))

# a = df1[df1['visuals']==vis[1]]['compPerc']
# b = df2[df2['visuals']==vis[2]]['compPerc']
# c = vis[1]
# d = vis[2]
# print(fancyMWU(a,b,c,d))

# a = df1[df1['visuals']==vis[1]]['compPerc']
# b = df2[df2['visuals']==vis[3]]['compPerc']
# c = vis[1]
# d = vis[3]
# print(fancyMWU(a,b,c,d))

# a = df1[df1['visuals']==vis[2]]['compPerc']
# b = df2[df2['visuals']==vis[3]]['compPerc']
# c = vis[2]
# d = vis[3]
# print(fancyMWU(a,b,c,d))
vis = ["Charts", "Blur+Charts", "Video", "Blurred"]

for v1 in vis:
    for v2 in vis:
        a = df1[df1['visuals']==v1]['comp_correctedPerc']
        b = df2[df2['visuals']==v2]['comp_correctedPerc']
        c = v1
        d = v2
        print(fancyMWU(a,b,c,d))

             Blur+Charts   Blurred    Charts     Video
Blur+Charts     1.000000  0.275822  0.119403  0.718280
Blurred         0.275822  1.000000  0.000639  0.718280
Charts          0.119403  0.000639  1.000000  0.028243
Video           0.718280  0.718280  0.028243  1.000000


Charts 51.61290322580645 37.60290540136035
Charts 37.03703703703704 32.79479494521311
MannwhitneyuResult(statistic=507.5, pvalue=0.13432992220881912)


Charts 51.61290322580645 37.60290540136035
Blur+Charts 50.0 37.416573867739416
MannwhitneyuResult(statistic=412.5, pvalue=0.8767911212319253)


Charts 51.61290322580645 37.60290540136035
Video 50.0 37.79644730092272
MannwhitneyuResult(statistic=460.0, pvalue=0.8738023198225382)


Charts 51.61290322580645 37.60290540136035
Blurred 48.214285714285715 37.22340675679768
MannwhitneyuResult(statistic=455.5, pvalue=0.7317220989301289)


Blur+Charts 30.357142857142858 34.26283407632531
Charts 37.03703703703704 32.79479494521311
MannwhitneyuResult(statistic=333.5, pvalue=0.

In [34]:
# Part Two:

df = likertSlim.copy()
# Wilcoxon Signed-Rank Test on likert1Mode v likert2Mode
a = 'likert1Mode_Z'
b = 'likert2Mode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likert1Mode v CompResponseMode
a = 'likert1Mode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likert2Mode v CompResponseMode
a = 'likert2Mode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likertMode v CompResponseMode
a = 'likertMode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))


for v in df.visuals.unique():
    print("\n>>>>>", v)
    tmpDF = df[df['visuals']==v]
    
    # Wilcoxon Signed-Rank Test on likert1Mode v likert2Mode
    a = 'likert1Mode_Z'
    b = 'likert2Mode_Z'
    print(fancyWilcoxon(a,b,tmpDF))


    # Wilcoxon Signed-Rank Test on likert1Mode v CompResponseMode
    a = 'likert1Mode_Z'
    b = 'CompResponseMode_Z'
    print(fancyWilcoxon(a,b,tmpDF))


    # Wilcoxon Signed-Rank Test on likert2Mode v CompResponseMode
    a = 'likert2Mode_Z'
    b = 'CompResponseMode_Z'
    print(fancyWilcoxon(a,b,tmpDF))


    # Wilcoxon Signed-Rank Test on likertMode v CompResponseMode
    a = 'likertMode_Z'
    b = 'CompResponseMode_Z'
    print(fancyWilcoxon(a,b,tmpDF))


likert1Mode_Z -0.3338589258904705 0.7058491661777792
likert2Mode_Z -0.17576746215945604 0.6980417634053117
WilcoxonResult(statistic=1854.5, pvalue=0.014584015178203067)

likert1Mode_Z -0.3338589258904705 0.7058491661777792
CompResponseMode_Z 0.3257762383066078 0.848205904356497
WilcoxonResult(statistic=2996.5, pvalue=6.107569715256834e-10)

likert2Mode_Z -0.17576746215945604 0.6980417634053117
CompResponseMode_Z 0.3257762383066078 0.848205904356497
WilcoxonResult(statistic=3222.0, pvalue=2.8495900696661057e-07)

likertMode_Z -0.3180010568577412 0.69451028115823
CompResponseMode_Z 0.3257762383066078 0.848205904356497
WilcoxonResult(statistic=3011.5, pvalue=1.1737795995425093e-09)

>>>>> Blur+Charts

likert1Mode_Z -0.3933804909271088 0.7748159219272214
likert2Mode_Z -0.1700381897993268 0.7511849304577585
WilcoxonResult(statistic=92.0, pvalue=0.09742223617083687)

likert1Mode_Z -0.3933804909271088 0.7748159219272214
CompResponseMode_Z 0.4003155030794404 0.694570642389455
WilcoxonResult(s

In [36]:
# Part Two:

print("EASY")

df = likertSlim[likertSlim['difficulty']=="Easy"].copy()

# Wilcoxon Signed-Rank Test on likert1Mode v likert2Mode
a = 'likert1Mode_Z'
b = 'likert2Mode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likert1Mode v CompResponseMode
a = 'likert1Mode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likert2Mode v CompResponseMode
a = 'likert2Mode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likertMode v CompResponseMode
a = 'likertMode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))



EASY

likert1Mode_Z -0.4708204106603042 0.6503880138607397
likert2Mode_Z -0.18335654532038273 0.7242353812560887
WilcoxonResult(statistic=363.0, pvalue=0.001804364638640656)

likert1Mode_Z -0.4708204106603042 0.6503880138607397
CompResponseMode_Z 0.4439448479613183 0.8104444062690473
WilcoxonResult(statistic=394.0, pvalue=4.087322354289558e-09)

likert2Mode_Z -0.18335654532038273 0.7242353812560887
CompResponseMode_Z 0.4439448479613183 0.8104444062690473
WilcoxonResult(statistic=739.5, pvalue=0.0001091743211628129)

likertMode_Z -0.3637213341071099 0.6857252676152094
CompResponseMode_Z 0.4439448479613183 0.8104444062690473
WilcoxonResult(statistic=596.0, pvalue=9.028931685147021e-07)


In [37]:
# Part Two:

print("Difficult")

df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Wilcoxon Signed-Rank Test on likert1Mode v likert2Mode
a = 'likert1Mode_Z'
b = 'likert2Mode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likert1Mode v CompResponseMode
a = 'likert1Mode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likert2Mode v CompResponseMode
a = 'likert2Mode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))


# Wilcoxon Signed-Rank Test on likertMode v CompResponseMode
a = 'likertMode_Z'
b = 'CompResponseMode_Z'
print(fancyWilcoxon(a,b,df))



Difficult

likert1Mode_Z -0.21036906257340743 0.7332601668309562
likert2Mode_Z -0.16892484619468606 0.6764833947934498
WilcoxonResult(statistic=560.0, pvalue=0.7739662177059887)

likert1Mode_Z -0.21036906257340743 0.7332601668309562
CompResponseMode_Z 0.21923077058514753 0.8704484969282394
WilcoxonResult(statistic=1132.0, pvalue=0.0035866181087289557)

likert2Mode_Z -0.16892484619468606 0.6764833947934498
CompResponseMode_Z 0.21923077058514753 0.8704484969282394
WilcoxonResult(statistic=904.5, pvalue=0.0009619084785483982)

likertMode_Z -0.2767778560591302 0.7025951965545549
CompResponseMode_Z 0.21923077058514753 0.8704484969282394
WilcoxonResult(statistic=921.5, pvalue=0.00019153531001294322)


In [45]:
# Get mean and sd of likerts

df = likertSlim.copy()

print("Part One Likerts")
print("mean: ", df['likert1Mode_Z'].mean())
print("std: ", df['likert1Mode_Z'].std())

print("\nPart Two Likerts")
print("mean: ", df['likert2Mode_Z'].mean())
print("std: ", df['likert2Mode_Z'].std())

print("\nPart One and Two Likerts")
print("mean: ", df['likertMode_Z'].mean())
print("std: ", df['likertMode_Z'].std())

print("\nPart Three Likerts")
print("mean: ", df['CompResponseMode_Z'].mean())
print("std: ", df['CompResponseMode_Z'].std())

print("\nAll Likerts")
print("mean: ", df['likertMode_All_Z'].mean())
print("std: ", df['likertMode_All_Z'].std())

Part One Likerts
mean:  -0.3338589258904705
std:  0.7058491661777792

Part Two Likerts
mean:  -0.17576746215945604
std:  0.6980417634053117

Part One and Two Likerts
mean:  -0.3180010568577412
std:  0.69451028115823

Part Three Likerts
mean:  0.3257762383066078
std:  0.848205904356497

All Likerts
mean:  0.04830488985769108
std:  0.6453217773116232


In [39]:
likertSlim.columns.tolist()

['ParticipantPrivateID',
 'difficulty',
 'visuals',
 'agents',
 'QsTotal',
 'QsCorrect',
 'Agent_QsTotal',
 'Agent_QsCorrect',
 'Agent_QsWO_GhostsTotal',
 'Agent_QsWO_GhostsCorrect',
 'Qs1_Correct',
 'Qs2_Correct',
 'QsPart_Total',
 'predictionsCorrect',
 'predictionsTotal',
 'predictions1Correct',
 'predictions2Correct',
 'predictionsHalfTotal',
 'regionsCorrect',
 'regionsTotal',
 'regions1Correct',
 'regions2Correct',
 'regionsHalfTotal',
 'levelCorrect',
 'levelTotal',
 'level1Correct',
 'level2Correct',
 'levelHalfTotal',
 'ghostsCorrect',
 'ghostsTotal',
 'ghosts1Correct',
 'ghosts2Correct',
 'ghostsHalfTotal',
 'comparisonsCorrect',
 'comparisonsTotal',
 'timeTakenMain_All',
 'timeTakenMain_AgentsOnly',
 'timeTaken_Consent',
 'timeTaken_Demographics',
 'timeTaken_AI',
 'timeTaken_Agent1',
 'timeTaken_Agent2',
 'timeTaken_Agent3',
 'time_TOTAL',
 'vidPlayed',
 'avgVidPlays',
 'ratioVidPlaysToAvg',
 'ExperimentVersion_x',
 'totPerc',
 'agTotPerc',
 'tot1Perc',
 'tot2Perc',
 'Agent

In [40]:
df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

vis = sorted(df.visuals.unique())

# Kruskal-Wallis on ALL compPerc across visuals
print("ALL 1 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['likertMode_Z'],
               df[df['visuals']==vis[1]]['likertMode_Z'],
               df[df['visuals']==vis[2]]['likertMode_Z'],
               df[df['visuals']==vis[3]]['likertMode_Z'])))

# Kruskal-Wallis on ALL compPerc across visuals
print("ALL 1 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['likert1Mode_Z'],
               df[df['visuals']==vis[1]]['likert1Mode_Z'],
               df[df['visuals']==vis[2]]['likert1Mode_Z'],
               df[df['visuals']==vis[3]]['likert1Mode_Z'])))

# Kruskal-Wallis on ALL compPerc across visuals
print("ALL 1 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['likert2Mode_Z'],
               df[df['visuals']==vis[1]]['likert2Mode_Z'],
               df[df['visuals']==vis[2]]['likert2Mode_Z'],
               df[df['visuals']==vis[3]]['likert2Mode_Z'])))

# Kruskal-Wallis on ALL compPerc across visuals
print("ALL 1 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['likertMode_All_Z'],
               df[df['visuals']==vis[1]]['likertMode_All_Z'],
               df[df['visuals']==vis[2]]['likertMode_All_Z'],
               df[df['visuals']==vis[3]]['likertMode_All_Z'])))

# Kruskal-Wallis on ALL compPerc across visuals
print("ALL 1 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['CompResponseMode_Z'],
               df[df['visuals']==vis[1]]['CompResponseMode_Z'],
               df[df['visuals']==vis[2]]['CompResponseMode_Z'],
               df[df['visuals']==vis[3]]['CompResponseMode_Z'])))

# Kruskal-Wallis on ALL compPerc across visuals
print("ALL 1 across visual")
print("And kw: " + \
str(stats.kruskal(df[df['visuals']==vis[0]]['likertMode_AndComp_Z'],
               df[df['visuals']==vis[1]]['likertMode_AndComp_Z'],
               df[df['visuals']==vis[2]]['likertMode_AndComp_Z'],
               df[df['visuals']==vis[3]]['likertMode_AndComp_Z'])))

ALL 1 across visual
And kw: KruskalResult(statistic=1.4424843092503687, pvalue=0.6956070805377281)
ALL 1 across visual
And kw: KruskalResult(statistic=1.6329322860342184, pvalue=0.6519459108720265)
ALL 1 across visual
And kw: KruskalResult(statistic=0.036907942427126444, pvalue=0.9981349307691288)
ALL 1 across visual
And kw: KruskalResult(statistic=3.5548906367905992, pvalue=0.31371245924202235)
ALL 1 across visual
And kw: KruskalResult(statistic=2.731469239464695, pvalue=0.4349058898821535)
ALL 1 across visual
And kw: KruskalResult(statistic=3.5548906367905992, pvalue=0.31371245924202235)
